In [7]:
!pip install matplotlib
!pip install seaborn


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 1.5 MB/s eta 0:00:000:00:010:00:01:01

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('jakarta_street_coordinate.csv')

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df = df[ ~df['coordinate'].isna() ].reset_index(drop=True)

In [5]:
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG JATI INDAH,"[-6.2599631, 106.773803]"
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG MASRIKI,"[-6.2626431, 106.7741125]"
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,H SAIMAN,"[-6.2614855, 106.7741336]"
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JATI INDAH,"[-6.2599631, 106.773803]"
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL BANK EXIM,"[-6.263339, 106.7745935]"


In [6]:
len(df)

25956

https://plotly.com/python/scatter-plots-on-maps/

We know that when we searched for the coordinate for the roads, we missed a lot of rows. However, we need those coordinates and we cannot afford any rows with missing data and thus removed them.

Let's do gap analysis for the _kelurahan_. That is, by checking how many _kelurahan_ have little or close to zero rows after we dropped all rows that weren't geocoded by Nominatim

In [7]:
df_all = pd.read_csv('NJOP2021.csv')
df_all = df_all.drop('Unnamed: 0', axis=1)
len(df_all)

219439

In [8]:
df_all = df_all.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()
df_all.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 001 - PONDOK PINANG,1382
1,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 002 - KEBAYORAN LAMA SLT,633
2,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 003 - KEBAYORAN LAMA UTR,725
3,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 004 - C I P U L I R,652
4,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 005 - GROGOL SELATAN,1141


In [9]:
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df_all[col] = df_all[col].apply(lambda x: str(x).split(' - ')[1])
df_all.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,1382
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SLT,633
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA UTR,725
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,C I P U L I R,652
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,GROGOL SELATAN,1141


In [10]:
df_agg = df.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()

In [11]:
df_merged = df_all.merge(df_agg, on=['provinsi', 'kota', 'kecamatan', 'kelurahan'], how='outer', suffixes=('_all', '_coord'))
df_merged.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan_all,nama_jalan_coord
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,1382,329.0
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SLT,633,NaN
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA UTR,725,NaN
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,C I P U L I R,652,NaN
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,GROGOL SELATAN,1141,221.0


In [12]:
df_merged['nama_jalan_coord'] = df_merged['nama_jalan_coord'].fillna(0)

In [13]:
df_merged['pct_coord'] = df_merged['nama_jalan_coord']/df_merged['nama_jalan_all']*100

In [14]:
df_merged.sort_values('pct_coord').head(40)

,provinsi,kota,kecamatan,kelurahan,nama_jalan_all,nama_jalan_coord,pct_coord
42,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN BARU,P U L O,224,0.0,0.000000
38,DKI JAKARTA,JAKARTA SELATAN,P A N C O R A N,C I K O K O,121,0.0,0.000000
39,DKI JAKARTA,JAKARTA SELATAN,P A N C O R A N,P A N C O R A N,275,0.0,0.000000
17,DKI JAKARTA,JAKARTA SELATAN,PASAR MINGGU,R A G U N A N,375,0.0,0.000000
47,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN BARU,S E L O N G,93,0.0,0.000000
49,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN BARU,S E N A Y A N,117,0.0,0.000000
53,DKI JAKARTA,JAKARTA SELATAN,SETIA BUDI,K A R E T,190,0.0,0.000000
56,DKI JAKARTA,JAKARTA SELATAN,SETIA BUDI,G U N T U R,260,0.0,0.000000
34,DKI JAKARTA,JAKARTA SELATAN,P A N C O R A N,KALI BATA,313,0.0,0.000000
29,DKI JAKARTA,JAKARTA SELATAN,MAMPANG PRAPATAN,B A N G K A,259,0.0,0.000000


We can see that some _kelurahan_ didn't even get any coordinate because of four main reasons:
1. `P U L O` should be written as `PULO`, the same problem goes until `C I P U L I R`
2. `UTR` should be written as `Utara` and `SLT` as `SELATAN`
3. `PAL MERIAM`, `KALI ANYAR`, `RAWA JATI`, `SETIA BUDI` should be written without space
4. `BALIMESTER` should be written as `BALI MESTER`

We'll focus on repairing those critical weak points using our geocode

In [15]:
reparse = df_merged.sort_values('pct_coord').head(21)['kelurahan'].to_list()

In [16]:
reparse.remove('HARAPAN MULIA')
reparse.remove('BIDARA CINA')

In [17]:
df_patch = pd.read_csv('./NJOP2021.csv')
df_patch = df_patch[['provinsi', 'kota', 'kecamatan', 'kelurahan', 'nama_jalan']]
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df_patch[col] = df_patch[col].apply(lambda x: str(x).split(' - ')[1])
df_patch = df_patch.drop_duplicates(ignore_index=True)
df_patch.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG FLAMBOYAN
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H MASRIKI
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAEMIN
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAIMIN
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H. SAIMIN


In [18]:
df_patch = df_patch[ df_patch['kelurahan'].isin(reparse) ]

In [19]:
remove_spaces = ['P U L O', 'C I K O K O', 'P A N C O R A N', 'R A G U N A N', 'S E L O N G', 'S E N A Y A N', 'K A R E T', 'G U N T U R', 'KALI BATA', 'B A N G K A', 'U L U J A M I', 'C I P U L I R', 'PAL MERIAM', 'KALI ANYAR', 'RAWA JATI', 'SETIA BUDI']
df_patch['kelurahan'] = df_patch['kelurahan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df_patch['kelurahan'] = df_patch['kelurahan'].str.replace('KEBAYORAN LAMA UTR', 'KEBAYORAN LAMA UTARA').str.replace('KEBAYORAN LAMA SLT', 'KEBAYORAN LAMA SELATAN').str.replace('BALIMESTER', 'BALI MESTER')

In [20]:
df_patch = df_patch.drop_duplicates(ignore_index=True)
df_patch.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,BUNGUR RAYA
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,GG H MI'AH
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,JL ALTERI
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,JL ARTERI
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA SELATAN,JL ARTERI PONDOK PINANG


In [21]:
from geopy.geocoders import Nominatim # https://nominatim.org/
import time

In [22]:
dict = {}
for idx in range(len(df_patch)):
    print('Now on index', idx)
    try:
        geolocator = Nominatim(user_agent="yusuf-application")
        location = geolocator.geocode(df_patch.iloc[idx]['nama_jalan']+', '+df_patch.iloc[idx]['kelurahan'], timeout=10)
        if location:
            dict[idx] = [location.latitude, location.longitude]
    except:
        idx -= 1
        time.sleep(60) # wait for internet to reconnect

Now on index 0
Now on index 1
Now on index 2
Now on index 3
Now on index 4
Now on index 5
Now on index 6
Now on index 7
Now on index 8
Now on index 9
Now on index 10
Now on index 11
Now on index 12
Now on index 13
Now on index 14
Now on index 15
Now on index 16
Now on index 17
Now on index 18
Now on index 19
Now on index 20
Now on index 21
Now on index 22
Now on index 23
Now on index 24
Now on index 25
Now on index 26
Now on index 27
Now on index 28
Now on index 29
Now on index 30
Now on index 31
Now on index 32
Now on index 33
Now on index 34
Now on index 35
Now on index 36
Now on index 37
Now on index 38
Now on index 39
Now on index 40
Now on index 41
Now on index 42
Now on index 43
Now on index 44
Now on index 45
Now on index 46
Now on index 47
Now on index 48
Now on index 49
Now on index 50
Now on index 51
Now on index 52
Now on index 53
Now on index 54
Now on index 55
Now on index 56
Now on index 57
Now on index 58
Now on index 59
Now on index 60
Now on index 61
Now on index 62
No

In [23]:
df_patch['coordinate'] = None
for i in dict:
    df_patch['coordinate'].iloc[i] = dict[i]

In [24]:
df_patch = df_patch[ ~df_patch['coordinate'].isna() ]

In [25]:
df_patch['kelurahan'].value_counts()

kelurahan
KEBAYORAN LAMA UTARA      214
ULUJAMI                   173
CIPULIR                   145
KEBAYORAN LAMA SELATAN    137
KALIBATA                  113
BANGKA                    109
PANCORAN                  108
RAGUNAN                    93
PULO                       85
PALMERIAM                  77
RAWAJATI                   67
SELONG                     64
BALI MESTER                48
CIKOKO                     46
KARET                      46
SENAYAN                    40
GUNTUR                     29
SETIABUDI                  21
KALIANYAR                   2
Name: count, dtype: int64

The result is much better now

In [26]:
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG JATI INDAH,"[-6.2599631, 106.773803]"
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG MASRIKI,"[-6.2626431, 106.7741125]"
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,H SAIMAN,"[-6.2614855, 106.7741336]"
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JATI INDAH,"[-6.2599631, 106.773803]"
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL BANK EXIM,"[-6.263339, 106.7745935]"


In [27]:
df_total = pd.concat([df, df_patch], axis=0)

In [28]:
df_total = df_total.reset_index(drop=True)
df_total = df_total.loc[df_total.astype(str).drop_duplicates().index]
df_total = df_total.reset_index(drop=True)

In [29]:
df_total.to_csv('jakarta_street_coordinate_patched.csv', index=False)

In [32]:
df_agg = df_total.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()

In [33]:
df_all = pd.read_csv('NJOP2021.csv')
df_all = df_all.drop('Unnamed: 0', axis=1)
df_all = df_all.groupby(['provinsi', 'kota', 'kecamatan', 'kelurahan'])['nama_jalan'].aggregate('nunique').reset_index()
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df_all[col] = df_all[col].apply(lambda x: str(x).split(' - ')[1])
remove_spaces = ['P U L O', 'C I K O K O', 'P A N C O R A N', 'R A G U N A N', 'S E L O N G', 'S E N A Y A N', 'K A R E T', 'G U N T U R', 'KALI BATA', 'B A N G K A', 'U L U J A M I', 'C I P U L I R', 'PAL MERIAM', 'KALI ANYAR', 'RAWA JATI', 'SETIA BUDI']
df_all['kelurahan'] = df_all['kelurahan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df_all['kelurahan'] = df_all['kelurahan'].str.replace('KEBAYORAN LAMA UTR', 'KEBAYORAN LAMA UTARA').str.replace('KEBAYORAN LAMA SLT', 'KEBAYORAN LAMA SELATAN').str.replace('BALIMESTER', 'BALI MESTER')

In [36]:
df_merged = df_all.merge(df_agg, on=['provinsi', 'kota', 'kecamatan', 'kelurahan'], how='outer', suffixes=('_all', '_coord'))
df_merged['nama_jalan_coord'] = df_merged['nama_jalan_coord'].fillna(0)
df_merged['pct_coord'] = df_merged['nama_jalan_coord']/df_merged['nama_jalan_all']*100
df_merged.sort_values('pct_coord').head(40)


,provinsi,kota,kecamatan,kelurahan,nama_jalan_all,nama_jalan_coord,pct_coord
155,DKI JAKARTA,JAKARTA PUSAT,KEMAYORAN,HARAPAN MULIA,125.0,1,0.800000
211,DKI JAKARTA,JAKARTA BARAT,TAMBORA,KALIANYAR,116.0,2,1.724138
95,DKI JAKARTA,JAKARTA TIMUR,JATINEGARA,BIDARA CINA,231.0,4,1.731602
263,DKI JAKARTA,KEPULAUAN SERIBU,KEPULAUAN SERIBU UTARA,PULAU KELAPA,42.0,1,2.380952
236,DKI JAKARTA,JAKARTA UTARA,PADEMANGAN,PADEMANGAN BARAT,354.0,9,2.542373
218,DKI JAKARTA,JAKARTA BARAT,TAMBORA,JEMBATAN LIMA,232.0,8,3.448276
108,DKI JAKARTA,JAKARTA TIMUR,DUREN SAWIT,MALAKA SARI,749.0,33,4.405874
261,DKI JAKARTA,KEPULAUAN SERIBU,KEPULAUAN SERIBU UTARA,PULAU HARAPAN,44.0,2,4.545455
136,DKI JAKARTA,JAKARTA PUSAT,TANAH ABANG,KAMPUNG BALI,191.0,9,4.712042
259,DKI JAKARTA,JAKARTA UTARA,CILINCING,SEMPER BARAT,749.0,44,5.874499


### Self Doodle, Nominatim and OpenStreetMap Really Doesn't Have the Data in Their Database

Since there are only 27k streets with coordinate, and we have seen that there are some locations that weren't able to be geocoded because of weird abbreviation (e.g. _UTR_ for _UTARA_). I suspect we will find lots of those cases. Let's see whether this hypothesis is true

In [4]:
df = pd.read_csv('jakarta_street_coordinate.csv')
df = df.drop('Unnamed: 0', axis=1)

In [32]:
words_list = []
words_set = set({})
final_words_set = set({})
for name in df['nama_jalan'].to_list():
    for word in name.split(' '):
        words_list.append(word)
        words_set.add(word)
for word in words_set:
    if words_list.count(word) > 100: # to remove insignificant words
        final_words_set.add(word)

In [40]:
def check_word(df, word, col, target):
    df = df.copy(deep=True)
    df[word+'_check'] = df[col].apply(lambda x: word in x.split(' ')) # population are rows with the word
    df[target+'_check'] = df[target].apply(lambda x: False if x != x else True) # check nan in target column
    df = df[ df[word+'_check'] ]
    try: # there is probability no True value
        num_found = df[target+'_check'].value_counts()[True] 
    except:
        num_found = 0
    try: # there is probability no False value
        num_unfound = df[target+'_check'].value_counts()[False]
    except:
        num_unfound = 0
    return [num_found, num_unfound, ((num_found+1)*100)/(num_found+num_unfound+2)] # Laplace Smoothing to prevent error

In [41]:
check_word(df, 'GG.', 'nama_jalan', 'coordinate')

[56, 824, 6.462585034013605]

In [42]:
words_dict = {}
for word in final_words_set:
    words_dict[word] = check_word(df, word, 'nama_jalan', 'coordinate')

In [43]:
checked_words = pd.DataFrame.from_dict(words_dict, orient='index', columns=['found', 'unfound', 'pct_found'])

In [51]:
checked_words.sort_values('unfound', ascending=False).head(30)

,found,unfound,pct_found
JL,17763,71571,19.884481
GG,2862,22626,11.231856
I,1433,8970,13.781836
II,1954,7219,21.307902
RAYA,1644,6434,20.358911
H,85,5968,1.420314
KO,2,5808,0.051617
III,1528,4465,25.504587
KP,676,4143,14.042730
BLOK,50,4090,1.231289


There are many interesting information here:
1. _JL_ should be abbreviation for _Jalan_ (Street)
2. _GG_ should be abbreviation for _Gang_ (Alley)
3. I don't know what _KO_ is. Need to be checked
4. I don't know what _KP_ is. Need to be checked
5. _KAV_ should be abbreviation for _Kavling_ (Parcel of Land -- No real English translation, though)

In [52]:
def check_row_by_word(df, word):
    df = df.copy(deep=True)
    df['check_word'] = df['nama_jalan'].apply(lambda x: word in x.split(' '))
    return df[ df['check_word'] ]

In [ ]:
def print_geolocation_if_exist(street):
    geolocator = Nominatim(user_agent="yusuf-application")
    location = geolocator.geocode(street, timeout=10)
    if location:
        print([location.latitude, location.longitude])

Let's check whether using the non-abbreviated form improves the performance

In [80]:
check_row_by_word(df, 'BLK').head(20)

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate,check_word
77,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL P LEBAK LESTARI BLK,NaN,True
134,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL PINANG EMAS I BLK F-4 KO PU,NaN,True
202,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL PINANG PERAK III BLK V,NaN,True
472,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL ALAM SEGAR V BLK IV UW-15,NaN,True
473,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL ALAM SEGAR V BLK UW,NaN,True
508,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL ALAM SEGAR XI BLK IV,NaN,True
1083,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL KENCANA PERMAI II BLK SR 14,NaN,True
1136,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL SEKOLAH KENCANA BLK II,NaN,True
1258,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL DUTA INDAH III BLK TL PS-12,NaN,True
2143,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,KEBAYORAN LAMA UTR,JL TAMAN GANDARIA BLK E,NaN,True


In [ ]:
print_geolocation_if_exist('JL TAMAN GANDARIA, KEBAYORAN LAMA UTARA')

NameError: name 'print_geolocation_if_exist' is not defined